In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from skimage import io

# Load your TIFF image data
img = io.imread("your_tiff_file.tif")

# Assuming 'img' is your image data with shape (T, C, Y, X)

# Brightfield channel index
brightfield_channel = 0

# Trap width range
min_trap_width = 100
max_trap_width = 150

# Iterate through timepoints
for timepoint in range(img.shape[0]):
    # Inverted Average Intensity Projection (AIP) for the brightfield channel
    inv_aip = 1 - np.mean(img[timepoint, brightfield_channel], axis=0)

    # Fourier analysis
    fourier = np.fft.rfft(inv_aip)
    freqs = np.fft.rfftfreq(inv_aip.size)
    magnitudes = np.abs(fourier)

    # Filter frequency peaks
    min_freq = 1 / max_trap_width
    max_freq = 1 / min_trap_width
    freq_filter = (freqs >= min_freq) & (freqs <= max_freq)
    filtered_freqs = freqs[freq_filter]
    filtered_magnitudes = magnitudes[freq_filter]

    # Find highest peak within range --> trap width
    max_magnitude_idx = np.argmax(filtered_magnitudes)
    trap_width_freq = filtered_freqs[max_magnitude_idx]
    trap_width = int(1 / trap_width_freq)
    print(f"Timepoint {timepoint + 1}: Estimated Trap Width = {trap_width}")

    # 1D filtering with "trap width-kernel"
    ys = inv_aip[:-trap_width] + inv_aip[trap_width:]
    xs = 0.5 * trap_width + np.arange(len(ys))

    # Find peaks --> trap centers
    peaks, _ = find_peaks(ys, distance=0.8 * trap_width)
    peaks = peaks[::2]  # Only if trap spacing = trap width!
    trap_centers = xs[peaks]

    # Crop and save traps
    for i, trap_center in enumerate(trap_centers):
        margin = 10  # You can adjust the margin
        trap_start = max(0, int(np.floor(trap_center - 0.5 * trap_width)) - margin)
        trap_end = min(img.shape[-1], int(np.ceil(trap_center + 0.5 * trap_width)) + margin)
        trap_img = img[timepoint, :, :, trap_start:trap_end]

        # Save the trap as a TIFF file
        io.imsave(f"trap_timepoint{timepoint + 1}_trap{i + 1}.tif", trap_img)
